### **Testing Plant Disease Prediction Model**

### #1 Importing Libraries

In [ ]:
# importing the required libraries
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import cv2
import glob
import random
import math

## #2 Loading Model

In [ ]:
# loading the saved model
loaded_model = tf.keras.models.load_model("../trained_model/plant_disease_trained_model.keras")

## #3 Model Summary

In [ ]:
# getting the model summary
loaded_model.summary()

## #4 Initializing the Disease Classes (For Prediction)

In [5]:
# defining the class names
class_names = ['Apple___Apple_scab',
 'Apple___Black_rot',
 'Apple___Cedar_apple_rust',
 'Apple___healthy',
 'Blueberry___healthy',
 'Cherry_(including_sour)___Powdery_mildew',
 'Cherry_(including_sour)___healthy',
 'Corn_(maize)___Cercospora_leaf_spot Gray_leaf_spot',
 'Corn_(maize)___Common_rust_',
 'Corn_(maize)___Northern_Leaf_Blight',
 'Corn_(maize)___healthy',
 'Grape___Black_rot',
 'Grape___Esca_(Black_Measles)',
 'Grape___Leaf_blight_(Isariopsis_Leaf_Spot)',
 'Grape___healthy',
 'Orange___Haunglongbing_(Citrus_greening)',
 'Peach___Bacterial_spot',
 'Peach___healthy',
 'Pepper,_bell___Bacterial_spot',
 'Pepper,_bell___healthy',
 'Potato___Early_blight',
 'Potato___Late_blight',
 'Potato___healthy',
 'Raspberry___healthy',
 'Soybean___healthy',
 'Squash___Powdery_mildew',
 'Strawberry___Leaf_scorch',
 'Strawberry___healthy',
 'Tomato___Bacterial_spot',
 'Tomato___Early_blight',
 'Tomato___Late_blight',
 'Tomato___Leaf_Mold',
 'Tomato___Septoria_leaf_spot',
 'Tomato___Spider_mites Two-spotted_spider_mite',
 'Tomato___Target_Spot',
 'Tomato___Tomato_Yellow_Leaf_Curl_Virus',
 'Tomato___Tomato_mosaic_virus',
 'Tomato___healthy']

## #5 Visualizing Single Image of Test Set

In [ ]:
# for locating specific image paths
from pathlib import Path

# specifying the image path 
image_path = "../test_images/AppleCedarRust1.JPG"
image_name = Path(image_path).stem

# reading the image using OpenCV
img = cv2.imread(image_path)
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB) # convert BGR image to RGB

# displaying the RGB mage
plt.imshow(img)
plt.title("Original Image")
plt.xticks([])
plt.yticks([])
plt.show()

## #6 Testing Model on a Single Image

In [ ]:
# preprocessing the image
image = tf.keras.preprocessing.image.load_img(image_path, target_size=(128, 128))
input_arr = tf.keras.preprocessing.image.img_to_array(image)
input_arr = np.array([input_arr]) #convert single image to batch format
print(input_arr.shape)

In [ ]:
# making prediction on the input image and displaying the result
prediction = loaded_model.predict(input_arr)
prediction, prediction.shape

In [ ]:
# getting and displaying the index of the class with highest probability
result_indx = np.argmax(prediction)
print(result_indx)

In [ ]:
# function to extract the disease name from the image
import re

def format_label(text):
    # removing digits from the end of the string
    text = re.sub(r'\d+$', '', text)
    
    # inserting space before capital letters
    formatted = re.sub(r'(?<!^)(?=[A-Z])', '_', text)
    
    return formatted

### Predicting for a Single image

In [ ]:
# displaying result of disease prediction on the image
model_prediction = class_names[result_indx]
plt.imshow(img)
plt.suptitle(f"Actual Disease: {format_label(image_name)}")
plt.title(f"Predicted Disease: {model_prediction}")
plt.xticks([])
plt.yticks([])
plt.show()

## #7 Predicting for a random batch of images

In [ ]:
# importing additional libraries
from pathlib import Path

# function to visualize batch predictions
def visualize_batch_predictions(folder_path, model, class_names, num_images=9, columns=3):
    
    # select the images
    all_image_paths = glob.glob(folder_path)
    
    if not all_image_paths:
        print(f"No images found in {folder_path}")
        return

    # select images at random
    selected_paths = random.sample(all_image_paths, min(len(all_image_paths), num_images))

    # setting up the subplots
    rows = math.ceil(len(selected_paths) / columns)
    fig, axes = plt.subplots(rows, columns, figsize=(5 * columns, 5 * rows))
    
    # handling scenarios where only 1 image is selected
    if num_images == 1:
        axes = np.array([axes])
        
    axes_flat = axes.flatten()

    # looping through the selected images and processing the predictions
    for i, ax in enumerate(axes_flat):
        if i < len(selected_paths):
            path = selected_paths[i]
            img_name = Path(path).stem

            # reading the image for display
            img = cv2.imread(path)
            if img is None:
                print(f"Failed to load image: {path}")
                continue
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

            # preprocessing the image
            image_tensor = tf.keras.preprocessing.image.load_img(path, target_size=(128, 128))
            input_arr = tf.keras.preprocessing.image.img_to_array(image_tensor)
            input_arr = np.array([input_arr])

            # performing model prediction
            prediction = model.predict(input_arr, verbose=0)
            result_indx = np.argmax(prediction)
            model_prediction = class_names[result_indx]
            
            # Helper to format the actual label (assuming label is in filename)
            # You can replace this logic if your labels are stored differently
            actual_label = format_label(img_name)
            
            # plotting the results
            ax.imshow(img)

            # setting the title
            ax.set_title(f"Actual: {actual_label}\nPredicted: {model_prediction}", fontsize=12)
            ax.axis('off')
        else:
            # hiding empty subplots
            ax.axis('off')

    plt.tight_layout()
    plt.show()

In [ ]:
visualize_batch_predictions(
    folder_path="../test_images/*",
    model=loaded_model, 
    class_names=class_names,
    num_images=4,            
    columns=4
)